#Common Setting

In [96]:
import pandas as pd
import numpy as np
import os
import glob
import sys
import hashlib
import re
from datetime import datetime
import time

import xml.etree.ElementTree as ET


In [97]:
start_time = datetime.now()
mode = "run" #run or test
env = 0 # 1- google colab,  else - local
if(env == 1):
  pathToDxml = "/content/drive/MyDrive/security_mft/dfxml_files"
else:
  pathToDxml = "."



In [98]:
# def getTag(tree):
#   tree = ET.parse('first_second.dfxml')
#   tagList = []
#   for elem in tree.iter():
#     tagList.append(re.sub(r"\{[^()]*\}", "", elem.tag))
#   #tagList = list(set(tagList))
#   return tagList

In [99]:
#test 1
s = '{http://www.forensicswiki.org/wiki/Category:Digital_Forensics_XML}gid'
s = re.sub(r'{.+}', '', s)
print(f"after: {s}")

after: gid


#Method 1 ElementTree

In [100]:
import re

# #test 1
# s = '{http://www.forensicswiki.org/wiki/Category:Digital_Forensics_XML}gid'
# s = re.sub(r'{.+}', '', s)
# print(f"after: {s}")

# #test 2
# s = '<@ """@$ FSDF >something something <more noise>'
# s = re.sub(r'<.+>', '',s)
# # s = re.sub('<[^>]+>', '', s)
# print(f"after: {s}")

def rename(s):
  s = re.sub(r'{.+}', '', s)
  # print(s)
  return s

In [101]:
def updt(total, progress):
    """
    Displays or updates a console progress bar.

    Original source: https://stackoverflow.com/a/15860757/1391441
    """
    barLength, status = 20, ""
    progress = float(progress) / float(total)
    if progress >= 1.:
        progress, status = 1, "\r\n"
    block = int(round(barLength * progress))
    text = "\r[{}] {:.0f}% {}".format(
        "#" * block + "-" * (barLength - block), round(progress * 100, 0),
        status)
    sys.stdout.write(text)
    sys.stdout.flush()

'''
runs = 300
for run_num in range(runs):
    time.sleep(.1)
    updt(runs, run_num + 1)
'''

'\nruns = 300\nfor run_num in range(runs):\n    time.sleep(.1)\n    updt(runs, run_num + 1)\n'

In [102]:
setDuplicate = set()
parentNodeInfo = dict()
def dictAssign(tempDict, key, item):
  # print(f"{tempDict}")
  # print(f"{key}")
  # print(f"{item}")
  global setDuplicate
  global parentNodeInfo
  nodeIter = 1
  #if duplicate
  '''
  if(key != "fileobject_id"):
    if(f"{key}_{parentNodeInfo[key]}" in tempDict.keys()):
      nodeIter = nodeIter + 1
      strSpecify = f"{key}_{parentNodeInfo[key]}_{nodeIter}"
      tempDict[strSpecify] = item
      return tempDict, nodeIter
  '''
  if(key in tempDict):
    while True:
      if(f"{key}_{nodeIter}" in tempDict):
        nodeIter = nodeIter+1
      else:
        break
    # if(tempDict[key]):
    setDuplicate.add(key)
    # print(parentNodeInfo)
    # nodeIter = nodeIter + 1
    strFirst = f"{key}_{nodeIter}"
    strSpecify = f"{key}_{nodeIter}"
    # tempDict[strFirst] = tempDict[key]
    tempDict[strSpecify] = item

    '''
      #store to a list
      if(type(tempDict[key]) is list):
        tempDict[key].append(item)
        strFirst = f"{key}_first"
        strSpecify = f"{key}_{parentNodeInfo}"
        tempDict[strFirst] = tempDict[key][0]
        tempDict[strSpecify] = item
      else:
        firstValue = tempDict[key]
        tempDict[key] = [firstValue]
        tempDict[key].append(item)
    '''
    '''
    else:
      #first time, not duplicate
      tempDict[key] = item
    '''
  else:
    tempDict[key] = item
    nodeIter = 0

  return tempDict, nodeIter

In [103]:




def showInfo_lxml(root):
  
  test = 0
  listColumns = ['fileobject_id']
  df = pd.DataFrame(columns=listColumns)
  byteRunDf = pd.DataFrame(columns=listColumns)
  fileobjects = root.findall('.//{http://www.forensicswiki.org/wiki/Category:Digital_Forensics_XML}fileobject')
  for fileobject_id, fileobject in enumerate(fileobjects):
    tempDict = {}
    byteRunTempDict = {}
    # tempDict['fileobject_id'] = fileobject_id
    
    tempDict = dictAssign(tempDict,"fileobject_id",fileobject_id)[0]
    
    for node in fileobject.iter():
      if(rename(node.tag) == "byte_run"):
        byteRunTempDict["fileobject_id"] = fileobject_id
        byteRunTempDict[rename(node.tag)] = node.text

        for key, item in node.attrib.items():
          byteRunTempDict[rename(key)] = item
        byteRunDf = byteRunDf.append(byteRunTempDict, ignore_index=True)

      else:
        global parentNodeInfo
        if(node.getparent()):
          # if(key in tempDict.keys())
          parentNodeInfo[rename(node.tag)] = rename(node.getparent().tag)
        else:
          parentNodeInfo[rename(node.tag)] = None

        # if(mode == 'test'):
        #   pass
          # if(test < 1):
          #   print(f"id: {fileobject_id}, tag: {rename(node.tag)}")
          #   print(f"id: {fileobject_id}, text: {node.text}")
          #   print(f"id: {fileobject_id}, attributes: {node.attrib}")
        # print(rename(node.tag))
        # tempDict[rename(node.tag)] = node.text
        tempDict, nodeIter = dictAssign(tempDict,rename(node.tag),node.text)

        # tempDict['attrib'] = node.attrib
        if(node.attrib): #if the attrib exists
          for key, item in node.attrib.items():
            # print(f"attrib: key: {key}, item: {item}")
            # tempDict = dictAssign(tempDict,rename(key),item)
            if(nodeIter != 0):
              tempDict[f'{rename(key)}_{rename(node.tag)}_{nodeIter}'] = item
            else:
              tempDict[f'{rename(key)}_{rename(node.tag)}'] = item
    df = df.append(tempDict, ignore_index=True)
    


    test+=1

    updt(len(fileobjects), fileobject_id + 1)
    if(mode == 'test'):
      if(test>20):
        print(f"fileobject 0 ~ {fileobject_id} is done. Then break...")
        break

  return df, byteRunDf
      





```
# This is formatted as code
```

# Method 2: lxml (can access parent node)

In [104]:
from lxml import etree


In [105]:
def parseDxml_lxml(pathToFile):
  start_time = datetime.now()

  tree = etree.parse(pathToFile)
  root = tree.getroot()
  df, byteRunDf = showInfo_lxml(root)

  csv_filename = f"{pathToDxml}/parse_result_{os.path.basename(pathToFile)[:-6]}.csv"
  csv_filename_byte_run = f"{pathToDxml}/parse_result_{os.path.basename(pathToFile)[:-6]}_byte_runs.csv"
  df.to_csv (csv_filename, index = False, header=True)
  byteRunDf.to_csv (csv_filename_byte_run, index = False, header=True)

  end_time = datetime.now()
  print(f"Duration of parsing data for {os.path.basename(pathToFile)[:-6]}: {end_time - start_time}")

In [106]:
test = 0
for eachFile in glob.glob(f"{pathToDxml}/*.dfxml"):
  print(f"parsing {eachFile} ...")
  # if (os.path.exists(f"{pathToDxml}/parse_result_{os.path.basename(eachFile)[:-6]}.csv")):
  #   print(f"{pathToDxml}/parse_result_{os.path.basename(eachFile)[:-6]}.csv exists, move to next dfxml file.")
  #   continue
  
  # print(f"size (bytes): {os.path.getsize(eachFile)}")
  # print(f"basename (no extention): {os.path.basename(eachFile)[:-6]}")
  parseDxml_lxml(eachFile)
  test+=1
  if(mode == "test"):
    if(test==0):
      break
print(f"the following columns has duplicated values: {setDuplicate}")
print(f"value of the len(setDuplicate): {len(setDuplicate)}")

parsing .\fifth_sixth.dfxml ...
[--------------------] 0% C:\Users\seanl\AppData\Local\Temp/ipykernel_137012/1168934919.py:26: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if(node.getparent()):
[####################] 100% 
Duration of parsing data for fifth_sixth: 0:03:04.463367
parsing .\first_second.dfxml ...
[####################] 100% 
Duration of parsing data for first_second: 0:01:33.508964
parsing .\fourth_fifth.dfxml ...
[####################] 100% 
Duration of parsing data for fourth_fifth: 0:00:04.265975
parsing .\second_third.dfxml ...
[####################] 100% 
Duration of parsing data for second_third: 0:00:46.704118
parsing .\sixth_seventh.dfxml ...
[####################] 100% 
Duration of parsing data for sixth_seventh: 0:01:54.381035
parsing .\third_fourth.dfxml ...
[####################] 100% 
Duration of parsing data for third_fourth: 0:01:41.276541
the following columns has

In [107]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
if(mode == "test"):
  filepath = glob.glob(f"{pathToDxml}/parse_result_fourth_fifth.csv")[0]#"/content/drive/MyDrive/security_mft/dfxml_files/parse_result_fourth_fifth.csv"
else:
  filepath = glob.glob(f"{pathToDxml}/*.csv")[0]#"/content/drive/MyDrive/security_mft/dfxml_files/parse_result_fourth_fifth.csv"
print(filepath)
df = pd.read_csv(filepath)
print(f"How many columns: {len(df.columns)}")
print(f"column names: {df.columns}")
print(df.head(50))

                 NaN   
24                     NaN                         NaN   
25                     NaN                         NaN   
26                     NaN                         NaN   
27                     NaN                         NaN   
28                     NaN                         NaN   
29                     NaN                         NaN   
30                     NaN                         NaN   
31                     NaN                         NaN   
32                     NaN                         NaN   
33                     NaN                         NaN   
34                     NaN                         NaN   
35                     NaN                         NaN   
36                     NaN                         NaN   
37                     NaN                         NaN   
38                     NaN                         NaN   
39                     NaN                         NaN   
40                     NaN                      